In [1]:
pip install --user tf-nightly

Note: you may need to restart the kernel to use updated packages.


In [2]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [3]:
batch_size = 32
img_height = 288
img_width = 432

In [4]:
data_dir_train = "OneDrive/Escritorio/Practicas Frustacion Videojuegos/img_melspec_struct/train/"
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir_train,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 3979 files belonging to 2 classes.


In [5]:
data_dir_val = "OneDrive/Escritorio/Practicas Frustacion Videojuegos/img_melspec_struct/evaluate/"
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir_val,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 1326 files belonging to 2 classes.


In [6]:
data_dir_test = "OneDrive/Escritorio/Practicas Frustacion Videojuegos/img_melspec_struct/test/"
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir_test,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 1328 files belonging to 2 classes.


In [7]:
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

(32, 288, 432, 3)
(32,)


In [8]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [9]:
normalization_layer = layers.experimental.preprocessing.Rescaling(1./255)

In [10]:
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
# Notice the pixels values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image)) 

0.0 1.0


In [11]:
num_classes = 2

model = Sequential([
  layers.experimental.preprocessing.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dropout(0.5),
  layers.Dense(num_classes)
])

In [12]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rescaling_1 (Rescaling)      (None, 288, 432, 3)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 288, 432, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 144, 216, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 144, 216, 32)      4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 72, 108, 32)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 72, 108, 64)       18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 36, 54, 64)        0

In [19]:
#weights
from sklearn.utils import class_weight


from pprint import pprint
# recordemos que tenemos un batch_size=32 por lo que tenemos dividios los spectrogramas de 32 en 32, aunque eb 
# las ultimas iteraciones no tiene porque pasar esto, por lo que utilizamos len()
y_train = []
for i in train_ds:
    for count_batch in range(0,len(i.__getitem__(1).numpy())):
        #print(i.__getitem__(1))
        y_train.append(i.__getitem__(1).numpy()[count_batch])

my_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(y_train),
                                                 y_train)
class_weights = {0: my_weights[0], 1: my_weights[1]}
print(class_weights)
#callbacks
my_callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=10),
    tf.keras.callbacks.ModelCheckpoint(
        filepath='model.{epoch:02d}.h5', monitor='val_acc', verbose=0, save_best_only=False,
        save_weights_only=False, mode='max', save_freq='epoch', options=None)
]
#epochs
epochs=100

{0: 4.793975903614458, 1: 0.5582210998877666}


In [21]:
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs,
  class_weight=class_weights,
  callbacks=my_callbacks,
)

Epoch 1/100
125/125 [==============================] - 232s 2s/step - loss: 0.3889 - accuracy: 0.8082 - val_loss: 0.4721 - val_accuracy: 0.7647
Epoch 2/100
125/125 [==============================] - 240s 2s/step - loss: 0.3045 - accuracy: 0.8537 - val_loss: 0.3302 - val_accuracy: 0.8635
Epoch 3/100
125/125 [==============================] - 229s 2s/step - loss: 0.2537 - accuracy: 0.8748 - val_loss: 0.4124 - val_accuracy: 0.8318
Epoch 4/100
125/125 [==============================] - 235s 2s/step - loss: 0.2066 - accuracy: 0.9002 - val_loss: 0.3468 - val_accuracy: 0.8906
Epoch 5/100
125/125 [==============================] - 228s 2s/step - loss: 0.1774 - accuracy: 0.9196 - val_loss: 0.4086 - val_accuracy: 0.8462
Epoch 6/100
125/125 [==============================] - 237s 2s/step - loss: 0.1311 - accuracy: 0.9394 - val_loss: 0.4579 - val_accuracy: 0.9087
Epoch 7/100
125/125 [==============================] - 228s 2s/step - loss: 0.1256 - accuracy: 0.9447 - val_loss: 0.4127 - val_accuracy:

In [22]:
#PROBAMOS A COGER UN MODELO GUARDADO
model_new = tf.keras.models.load_model('model.12.h5')

test_lost, test_acc= model_new.evaluate(test_ds)
print ("Test Accuracy2:", test_acc)
print ("Test Lost2:", test_lost)

42/42 [==============================] - 21s 488ms/step - loss: 0.5553 - accuracy: 0.0896
Test Accuracy2: 0.08960843086242676
Test Lost2: 0.555250883102417


In [23]:
# PROBAMOS CON EL ULTIMO MODELO CARGADO
test_loss, test_acc= model.evaluate(test_ds)
print ("Test Accuracy:", test_acc)
print ("Test Loss:", test_loss)

42/42 [==============================] - 24s 577ms/step - loss: 0.5553 - accuracy: 0.8931
Test Accuracy: 0.8930723071098328
Test Loss: 0.5552509427070618


In [106]:
# por ultimo comprobamos nuestro porcentaje de acierto
test = model.predict(test_ds)

In [95]:
classes = np.argmax(test, axis = 1)
print(len(classes))
#for i in classes:
#    print(i)
#print(classes[62]) 

1328


In [104]:
from pprint import pprint
# recordemos que tenemos un batch_size=32 por lo que tenemos dividios los spectrogramas de 32 en 32, aunque eb 
# las ultimas iteraciones no tiene porque pasar esto, por lo que utilizamos len()
classes_predictions = []
for i in test_ds:
    for count_batch in range(0,len(i.__getitem__(1).numpy())):
        #print(i.__getitem__(1))
        classes_predictions.append(i.__getitem__(1).numpy()[count_batch])
            
print(len(classes_predictions))

1328


In [122]:
count_correct = 0
count_wrong = 0
countCorrectFrus = 0
for i in range(0,len(classes)):
    #vamos a crear un csv
    if classes[i] == 0 and classes_predictions[i] == 0:
        countCorrectFrus = countCorrectFrus+1
    if classes[i] != classes_predictions[i]:
        count_wrong = count_wrong+1
    else:
        count_correct = count_correct+1
print("total:", len(classes))
print("Acertadas:", count_correct)
print("Falladas:", count_wrong)
print("Acertadas Frustracion:",countCorrectFrus)

# seria como el accuracy???
print("Accuracy:", count_correct/len(classes))

total: 1328
Acertadas: 1100
Falladas: 228
Acertadas Frustracion: 8
Accuracy: 0.8283132530120482


In [76]:
df = pd.DataFrame({'Label':classes,
    'Predictions':classes_predictions})
df.to_excel("img_melspec_predictions_weight.xlsx") 